## 3D Plots

### 3D side by side

In [1]:
import os,sys
sys.path.append('..')

from matplotlib import pyplot as plt
import matplotlib as mpl
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from skimage.filters import gaussian
from skimage.transform import rescale
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from ridge_eval.ridge_extractor import open_surface_img

main_dir = Path(r"D:\Publications\Paper4-Wetting-ridge\test_data\mult")
plot_out_dir = main_dir / "level_compare"

plane_level_files = list((main_dir/"plane_level").glob("*.pkl.gz"))
polylevel_files = list((main_dir/"polylevel").glob("*.pkl.gz"))
polylevel3_files = list((main_dir/"polylevel3").glob("*.pkl.gz"))
flatten_base_files = list((main_dir/"flatten_base").glob("*.pkl.gz"))
nolevel_files = list((main_dir/"nolevel").glob("*.pkl.gz"))


images = {}
rescale_factor = 8
for nl, pl, ply, ply3, fb in zip(nolevel_files, plane_level_files, polylevel_files, polylevel3_files, flatten_base_files):
    # if not "80_S" in str(file):
    #     continue
    assert nl.stem == pl.stem == ply.stem == fb.stem
    nl_img, nl_px = open_surface_img(nl)
    pl_img, pl_px = open_surface_img(pl)   
    ply_img, ply_px = open_surface_img(ply)
    ply3_img, ply3_px = open_surface_img(ply3)
    fb_img, fl_px = open_surface_img(fb)

    fig = make_subplots(
        rows=1, cols=5, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0, horizontal_spacing=0, 
        specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]],
        subplot_titles=["No Leveling", "Plane Leveling", "Poly Leveling", "polyleveling deg3", "Flatten Base"]
    )

    def add_subplot(img, px, idx):
        img = rescale(img, 1/rescale_factor, anti_aliasing=True)
        # img -= img.mean()
        img *= 1000
        y = ((np.arange(img.shape[0])) * px[0] * rescale_factor)*1000
        x = ((np.arange(img.shape[1])) * px[1] * rescale_factor)*1000
        fig.add_trace(go.Surface(coloraxis="coloraxis", z=img, x=x,y=y ),
                        row=1, col=idx)
    
    add_subplot(nl_img, nl_px, 1)
    add_subplot(pl_img, pl_px, 2)
    add_subplot(ply_img, ply_px, 3)
    add_subplot(ply3_img, ply3_px, 4)
    add_subplot(fb_img, fl_px, 5)


    fig.update_layout(#title="80_S", autosize=True,
                        width=2400, height=600,
                        margin=dict(l=0, r=0, b=0, t=0),
                        font_size=12,
                        font_color="black",
                        font_family="Latin Modern",
    )

    fig.update_coloraxes(colorscale='icefire', colorbar=dict(len=1, thickness=10, orientation = "v", xanchor="left", x=0), 
                            colorbar_title="Surface Height <i>z</i> (mm)", 
                            colorbar_title_side="top", cmax=0.006, cmin=-0.006)


    scene=dict(aspectmode="manual", aspectratio=dict(x=1, y=1, z=0.2),
                xaxis_title="x (mm)",
                yaxis_title="y (mm)",
                # zaxis_title="Height (mm)",
                zaxis = dict(nticks=2,  showticklabels=False, showaxeslabels=False, title_text=""))#, range=[-0.0045,0.008]))

    fig.update_layout(scene1=scene, scene2=scene, scene3=scene, scene4=scene, scene5=scene)

    # fig.show()
    # fig.write_html(str(plot_out_dir / "figure2_3d_compare.html"))
    fig.write_image(str(plot_out_dir / nl.with_suffix(".png").name),scale=4)
    # break

DIPlib -- a quantitative image analysis library
Version 3.4.3 (Mar 21 2024)
For more information see https://diplib.org
